#**Alexander Bohlman**#
##Maryville University of St. Louis##
###Capstone Project###

#**Analysis and Insights Using Machine Learning Techniques on Air Pollution Data**#


##**Air pollution played a role in 10% of all deaths (about 1,600 people) in the Twin Cities metro in 2015.**##
*Air Quality Trends and Data. Minnesota Pollution Control Agency. (n.d.). https://www.pca.state.mn.us/air-water-land-climate/air-quality-trends-and-data*


###**Questions to Keep in Mind**###
What pollutants are most harmful?

How can we avoid Pollutants?

Is low air quality constant or occasional?


Can we label air quality accurately?


##Seoul dataset, real world collected data from stations in Seoul, South Korea##
*https://www.kaggle.com/datasets/bappekim/air-pollution-in-seoul*

##Air Quality dataset, air pollution data with pre-labeled air quality levels##
*https://www.kaggle.com/datasets/mujtabamatin/air-quality-and-pollution-assessment/data*

In [1]:
#Load required Libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
#Load required Libraries

In [2]:
#Load Pollution data
file_path = "/updated_pollution_dataset.csv"
air_data = pd.read_csv(file_path)
air_data.head()
#Load Pollution data

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,Moderate
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,Moderate
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,Moderate
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,Good
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,Good


#*Synthetic Data*#

#**Polluttants**#
PM2.5 (Particulate Matter 2.5 Micrometers) - Dust, smoke, soot, etc...

PM10 (Particulate Matter 10 Micrometers) - Larger Coarser Versions of the above.

NO2 (Nitrogen Dioxide) - A gas, caused by burning fuels, can irritate lungs, contributes to smog.

SO2 (Sulfur Dioxide) - A gas, caused by burning fuels, can irritate airways.

CO (Carbon Monoxide) - A gas, caused by incomplete burning of fuels, poisionus as it starves the blood of oxygen.

#*Temperature in Celcius, Humidity as a percentage, Proximity in km, Population Density as people/km^2*#


In [3]:
#Clean Data
air_data.rename(columns={'Proximity_to_Industrial_Areas': 'industry_Proximity', 'Air Quality': 'air_Quality','Population_Density': 'population_Density',
                         'Temperature': 'temp','Humidity': 'humidity'}, inplace=True) #Change column names for readibility
encoder = LabelEncoder()
scaler = StandardScaler()
target_names = ['Good', 'Moderate', 'Poor', 'Hazardous']
columns = ['industry_Proximity', 'population_Density', 'temp', 'humidity', 'PM2.5','PM10', 'SO2', 'CO', 'NO2']
air_data[columns] = scaler.fit_transform(air_data[columns])
air_data['air_Quality'] = encoder.fit_transform(air_data['air_Quality'])
air_data.drop_duplicates(inplace=True)
air_data.dropna(inplace=True)
air_data.head()
#Clean Data

,temp,humidity,PM2.5,PM10,NO2,SO2,CO,industry_Proximity,population_Density,air_Quality
0,-0.034080,-0.690715,-0.608589,-0.450455,-0.844581,-0.120721,0.402303,-0.588658,-1.168163,2
1,-0.257295,0.349507,-0.726706,-0.658892,0.493329,-0.046643,0.255775,-0.671748,0.743598,2
2,-1.031106,0.292768,0.267100,0.130973,-0.226219,0.383011,0.237459,-0.893318,0.795975,2
3,-0.435867,-1.951591,-0.571933,-0.874642,-1.451700,-0.698530,-0.641707,0.740767,0.350770,0
4,-0.525153,0.040593,-0.539349,-0.519934,-0.507293,-0.654083,-0.898130,1.183909,-1.272917,0


*   Modified column names to standardiz ethe naming convention
*   Reordered dataframe to bring record ID to the front
*   Dropped duplicates and nulls



In [4]:
air_data.info() #View column info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temp                5000 non-null   float64
 1   humidity            5000 non-null   float64
 2   PM2.5               5000 non-null   float64
 3   PM10                5000 non-null   float64
 4   NO2                 5000 non-null   float64
 5   SO2                 5000 non-null   float64
 6   CO                  5000 non-null   float64
 7   industry_Proximity  5000 non-null   float64
 8   population_Density  5000 non-null   float64
 9   air_Quality         5000 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 390.8 KB


In [5]:
air_data['air_Quality'].value_counts()

,count
air_Quality,
0,2000
2,1500
3,1000
1,500


5000 Total Records, all in the form of floats or objects after encoding the air qualities. The data is quite imbalanced, with the number of classifications in the data declining as the air quality got worse.
#**0 = Good, 2 = Moderate, 3 = Poor, 1 = Hazardous**#


In [6]:
label = air_data['air_Quality'] #Strip Air Quality as our prediction label
features = air_data.drop('air_Quality', axis=1) #Take the rest as prediction features

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=18) #strip test set from data

Air Quality is the label for classification, all other data will be used as features

In [8]:
#Run models to determine best option
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.919

In [9]:
#Run models to determine best option
model = KNeighborsClassifier(n_neighbors= 1)
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.918

In [10]:
#Run models to determine best option
model = RandomForestClassifier() #RandomForest Acheived a very high accuracy, this will be the selected model
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.951

Ran 3 different models, RandomForestClassifier was the best performing. As a result, this will be the model we go forward with.

In [11]:
#Tune hyperparemeters for RandomForest
forest_model = RandomForestClassifier(n_estimators = 100, random_state = 18)
forest_parameters = {'max_depth' : range(10, 40, 10)}
forest_search = GridSearchCV(forest_model, forest_parameters, cv=5, scoring='accuracy')
forest_search.fit(X_train, y_train)
print(f'Best Parameters: {forest_search.best_params_}')
print(f'Best Score: {forest_search.best_score_}')
#Tune hyperparemeters for RandomForest

Best Parameters: {'max_depth': 30}
Best Score: 0.95275


In [12]:
#Run with new parameter
forest_model = RandomForestClassifier(n_estimators = 100, random_state = 1, max_depth = 30)
forest_model.fit(X_train, y_train)
forest_model.score(X_test, y_test)
predictions = forest_model.predict(X_test)
print(classification_report(y_test, predictions))
#Run with new parameter

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       384
           1       0.97      0.85      0.90        99
           2       0.95      0.98      0.97       308
           3       0.91      0.91      0.91       209

    accuracy                           0.96      1000
   macro avg       0.96      0.94      0.95      1000
weighted avg       0.96      0.96      0.96      1000



The precision, recall, and f1-score, are all quite high and suggest a good understanding of the training. Overall the model was able to acheive an accuracy of 96% of predictions being correct.

In [13]:
#Explore what features carried the most weight
importances = forest_model.feature_importances_
feature_names = X_train.columns
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)
#Explore what features carried the most weight

              Feature  Importance
6                  CO    0.354323
7  industry_Proximity    0.290250
4                 NO2    0.097762
5                 SO2    0.089124
0                temp    0.065165
8  population_Density    0.035640
1            humidity    0.033207
3                PM10    0.021196
2               PM2.5    0.013333


The weights assigned to the features provide insights into what mattered for the model. Carbon Monoxide is the single biggest pollutant that impacts air quality. The other major factor is the environmental aspect of being physically closer to industry centers

In [14]:
#Download seoul air pollution data
file_path = "/Measurement_summary.csv"
seoul_data = pd.read_csv(file_path)
seoul_data.head()
#Download seoul air pollution data

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0


In [15]:
#Check the shapes to determine what is required to make them match
seoul_data.info()
air_data.info()
#Check the shapes to determine what is required to make them match

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163993 entries, 0 to 163992
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Measurement date  163993 non-null  object 
 1   Station code      163993 non-null  int64  
 2   Address           163993 non-null  object 
 3   Latitude          163993 non-null  float64
 4   Longitude         163993 non-null  float64
 5   SO2               163992 non-null  float64
 6   NO2               163992 non-null  float64
 7   O3                163992 non-null  float64
 8   CO                163992 non-null  float64
 9   PM10              163992 non-null  float64
 10  PM2.5             163992 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 13.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----

This new dataset is taken from stations around Seoul, South Korea. With nearly 650,000 records. It has identical pollutant information but lacks the environmental information that the original dataset had. As a result, we will only be keeping the pollutant information from each as we need the same shape in order to make predcitions.

In [16]:
#fit the data to new identical dataframes
seoul_air_shaped = seoul_data[['SO2','NO2','PM2.5','PM10', 'CO']]
seoul_columns = ['SO2','NO2','PM2.5','PM10', 'CO']
seoul_air_shaped[seoul_columns] = scaler.fit_transform(seoul_air_shaped[seoul_columns])
pollution_shaped = air_data[['SO2', 'NO2', 'PM2.5', 'PM10', 'CO', 'air_Quality']]
#fit the data to new identical dataframes

<ipython-input-16-8e23b4b28da8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_air_shaped[seoul_columns] = scaler.fit_transform(seoul_air_shaped[seoul_columns])


Create identical dataframes and standardize the new data.

In [17]:
#Split new dataset and create training, validation, and test sets
label = pollution_shaped['air_Quality']
features = pollution_shaped.drop('air_Quality', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=18)
#Split new dataset and create training, validation, and test sets

In [18]:
#Run models to determine best option
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.857

In [19]:
#Run models to determine best option
model = KNeighborsClassifier(n_neighbors= 1)
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.855

In [20]:
#Run models to determine best option
model = RandomForestClassifier() #RandomForest Acheived a very high accuracy, this will be the selected model
model.fit(X_train, y_train)
model.score(X_test, y_test)
#Run models to determine best option

0.902

In [21]:
#Tune hyperparemeters for RandomForest
forest_model = RandomForestClassifier(n_estimators = 100, random_state = 18)
forest_parameters = {'max_depth' : range(10, 50, 5)}
forest_search = GridSearchCV(forest_model, forest_parameters, cv=5, scoring='accuracy')
forest_search.fit(X_train, y_train)
print(f'Best Parameters: {forest_search.best_params_}')
print(f'Best Score: {forest_search.best_score_}')
#Tune hyperparemeters for RandomForest

Best Parameters: {'max_depth': 10}
Best Score: 0.9012499999999999


In [22]:
#Run RandomForest Model
seoul_forest_model = RandomForestClassifier(n_estimators = 100, random_state = 18, max_depth = 10)
seoul_forest_model.fit(X_train, y_train)
score  = seoul_forest_model.score(X_test, y_test)
print(score)
#Run RandomForest Model

0.905


In [23]:
#Analyze the performance
predictions = seoul_forest_model.predict(X_test)
print(classification_report(y_test, predictions))
#Run with new parameter

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       384
           1       0.87      0.79      0.83        99
           2       0.88      0.92      0.90       308
           3       0.81      0.79      0.80       209

    accuracy                           0.91      1000
   macro avg       0.89      0.87      0.88      1000
weighted avg       0.90      0.91      0.90      1000



Hazardous predicted, and record was Hazardous 87% (False Positive)
Hazardous not predicted, and record was Hazardous 79% (False Negative)


After running multiple models, randomForestClassifier was again the most efficient option, after tuning it the model achieved an over 90% accuracy. As we saw earlier the influence industry proximity had, as a result the model has lost some accuracy without the environmental factors.

In [24]:
#Explore what features carried the most weight
importances = seoul_forest_model.feature_importances_
feature_names = X_train.columns
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)
#Explore what features carried the most weight

  Feature  Importance
4      CO    0.536608
0     SO2    0.193063
1     NO2    0.171975
3    PM10    0.063659
2   PM2.5    0.034696


Without environmental factors CO has become an even more important weight, making up over 50% of the models decision making.

In [25]:
#Predict the Seoul data with the new model
predictions = seoul_forest_model.predict(seoul_air_shaped)
seoul_data['air_Quality'] = predictions
#Predict the Seoul data with the new model

In [26]:
#Group all data by station code
seoul_data.groupby('Station code').value_counts()
#Group all data by station code

Station code  Measurement date  Address                                                        Latitude   Longitude   SO2    NO2    O3     CO   PM10  PM2.5  air_Quality
101           2017-01-01 00:00  19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea      37.572016  127.005008  0.004  0.059  0.002  1.2  73.0  57.0   3              1
              2017-01-01 01:00  19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea      37.572016  127.005008  0.004  0.058  0.002  1.2  71.0  59.0   3              1
              2017-01-01 02:00  19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea      37.572016  127.005008  0.004  0.056  0.002  1.2  70.0  59.0   3              1
              2017-01-01 03:00  19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea      37.572016  127.005008  0.004  0.056  0.002  1.2  70.0  58.0   3              1
              2017-01-01 04:00  19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea      37.572016  127.005008  0.003  0.051  0.002  1.2  69.0  61.0   3              1
                                                                                                                                                                           ..
107           2017-12-24 14:00  18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea  37.541864  127.049659  0.005  0.026  0.017  0.5  5.0   3.0    2              1
              2017-12-24 15:00  18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea  37.541864  127.049659  0.005  0.021  0.017  0.6  7.0   9.0    2              1
              2017-12-24 16:00  18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea  37.541864  127.049659  0.004  0.020  0.021  0.6  14.0  25.0   2              1
              2017-12-24 17:00  18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea  37.541864  127.049659  0.005  0.018  0.024  0.5  21.0  19.0   2              1
              2017-12-24 18:00  18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea  37.541864  127.049659  0.005  0.020  0.021  0.6  27.0  32.0   2              1
Name: count, Length: 163992, dtype: int64

Use the model to predict the new dataset, classifying the air quality for every record of air polluntant data.

In [27]:
#Return to string labels
seoul_data['air_Quality'] = encoder.inverse_transform(seoul_data['air_Quality'])
seoul_data.head()
#Return to string labels

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,air_Quality
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,Poor
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0,Poor
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0,Poor
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0,Poor
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0,Poor


Undo encoding to return the original labels

In [28]:
#Take grouped data and determine counts of each "Non-Good" air quality event
air_quality_counts = seoul_data.groupby('Station code')['air_Quality'].value_counts().unstack(fill_value=0)
column_names = ['Moderate', 'Poor', 'Hazardous']
poor_aq_counts = air_quality_counts.reindex(columns=column_names, fill_value=0).astype(int)
station_info = seoul_data[['Station code', 'Latitude', 'Longitude']].drop_duplicates(subset=['Station code']).set_index('Station code')
station_data = station_info.join(poor_aq_counts, how='left')
station_data['Total'] = station_data[column_names].sum(axis=1)
station_data[column_names] = station_data[column_names].fillna(0).astype(int)
station_data = station_data.reset_index()
print("New DataFrame created from 'air_big_data':")
print(station_data)
#Take grouped data and determine counts of each "Non-Good" air quality event

New DataFrame created from 'air_big_data':
   Station code   Latitude   Longitude  Moderate  Poor  Hazardous  Total
0           101  37.572016  127.005008     21027  4467        356  25850
1           102  37.564263  126.974676     21831  3302        150  25283
2           103  37.540033  127.004850     23566  1983         53  25602
3           104  37.609823  126.934848     19085  4933        165  24183
4           105  37.593742  126.949679     19314  4534        552  24400
5           106  37.555580  126.905597     21706  3230        198  25134
6           107  37.541864  127.049659      7758   714          4   8476


Create a dataframe with the station information and the "non-good" air quality counts

In [29]:
station_data.to_csv('station_data.csv', index=False)